In [1]:
!pip install gpxpy

In [2]:
import os
import pandas as pd
import numpy as np
import json
import os
import gpxpy
from datetime import datetime
from dateutil import parser

In [3]:
workouts_folder_path1 = "/content/drive/MyDrive/Dane z aplikacji/Endomondo/Workouts"
workouts_folder_path2 = "/content/drive/MyDrive/Dane z aplikacji/MapMyRun/user184238914_workout_history.csv"
workouts_folder_path3 = "/content/drive/MyDrive/Dane z aplikacji/Sports Tracker/Running workouts"
workouts_folder_path4 = "/content/drive/MyDrive/Dane z aplikacji/Garmin Connect/Aktywności/Bieganie.csv"

**df1 => Endomondo export**

In [4]:
dfs = []

files = [f for f in os.listdir(workouts_folder_path1) if f.endswith(".json")]

if files:
    for file_name in files:
        file_path = os.path.join(workouts_folder_path1, file_name)

        with open(file_path, 'r') as file:
            try:
                data = json.load(file)
                if isinstance(data, list) and len(data) > 0:
                    sport = 'Running' if data[0].get('sport') == 'RUNNING' else data[0].get('sport')
                    if sport == 'Running':
                        source = data[1].get('source')
                        source = source.replace("INPUT_MANUAL", "Endomondo (Manual)").replace("TRACK_MOBILE", "Endomondo (GPS)")

                        start_time = pd.to_datetime(data[3].get('start_time'))
                        end_time = pd.to_datetime(data[4].get('end_time'))
                        speed_avg_kmh = round(float(data[8].get('speed_avg_kmh')), 2) if data[8].get('speed_avg_kmh') is not None else None
                        duration = (str(data[5].get('duration_s')).split('.')[0])if data[5].get('duration_s') is not None else None
                        distance = round(float(data[6].get('distance_km')), 2) if duration is not None else None

                        dfs.append({
                            'sport': sport,
                            'source': source,
                            'date': start_time.date(),
                            'start_time': start_time.time(),
                            'end_time': end_time.time(),
                            'duration_s': duration,
                            'distance_km': distance,
                            'speed_avg_kmh': speed_avg_kmh
                        })

            except Exception as e:
                print(f"Error processing JSON file {file_name}: {e}")

    df1 = pd.DataFrame(dfs).sort_values(by='date').reset_index(drop=True)
    df1['date'] = df1['date'].astype(str).str.strip()
    df1['date'] = pd.to_datetime(df1['date'])
    df1['distance_km'] = df1['distance_km'].astype(float).round(2)
    df1['duration_s'] = pd.to_numeric(df1['duration_s'], errors='coerce').astype('Int64')
    display(df1)
else:
    print("No JSON files found in the specified folder.")


,sport,source,date,start_time,end_time,duration_s,distance_km,speed_avg_kmh
0,Running,Endomondo (Manual),2012-08-15,10:07:00,10:33:00,1560,4.21,9.72
1,Running,Endomondo (Manual),2012-08-18,18:07:00,19:04:00,3420,8.48,8.93
2,Running,Endomondo (Manual),2012-08-20,18:30:00,19:19:00,2940,6.73,8.24
3,Running,Endomondo (Manual),2012-08-22,18:30:00,19:24:00,3240,7.77,8.63
4,Running,Endomondo (Manual),2012-08-26,18:21:00,19:13:00,3120,7.77,8.97
...,...,...,...,...,...,...,...,...
646,Running,Endomondo (GPS),2020-10-25,05:47:17,06:46:08,3522,10.07,NaN
647,Running,Endomondo (GPS),2020-10-29,14:59:15,16:00:49,3673,10.03,NaN
648,Running,Endomondo (GPS),2020-11-01,16:52:07,16:52:10,3239,9.44,NaN
649,Running,Endomondo (GPS),2020-11-04,19:53:52,19:53:55,3370,9.37,NaN


**df2 => Map My Run export**

In [5]:
df = pd.read_csv(workouts_folder_path2, header=None, skip_blank_lines=False)

df2 = df[df[2] == 'Run'].copy()

df2.rename(columns={1: 'date', 2: 'sport', 4: 'distance_km', 5: 'duration_s', 8: 'speed_avg_kmh', 13: 'source2', 14:'url'}, inplace=True)
df2['sport'] = df2['sport'].apply(lambda x: 'Running' if x == 'Run' else x)
df2['source'] = 'Map My Run app'
df2.drop(columns=[0, 3, 6, 7, 9, 10, 11, 12], inplace=True)

df2['date'] = df2['date'].apply(lambda x: parser.parse(x).strftime('%Y-%m-%d'))
df2['date'] = pd.to_datetime(df2['date'])
df2['distance_km'] = df2['distance_km'].astype(float).round(2)
df2['duration_s'] = df2['duration_s'].astype(int)
df2['speed_avg_kmh'] = df2['speed_avg_kmh'].astype(float).round(2)

df2 = df2[['sport', 'source', 'date', 'duration_s', 'distance_km', 'speed_avg_kmh']]

display(df2)


,sport,source,date,duration_s,distance_km,speed_avg_kmh
1,Running,Map My Run app,2020-12-17,600,1.61,9.66
3,Running,Map My Run app,2020-11-08,3163,9.16,10.43
4,Running,Map My Run app,2020-11-04,3370,9.37,10.01
7,Running,Map My Run app,2020-11-01,3239,9.44,10.49
11,Running,Map My Run app,2020-10-29,3673,10.03,9.83
...,...,...,...,...,...,...
4776,Running,Map My Run app,2012-08-26,3120,7.77,8.97
4777,Running,Map My Run app,2012-08-22,3240,7.77,8.63
4778,Running,Map My Run app,2012-08-20,2940,6.73,8.24
4779,Running,Map My Run app,2012-08-18,3420,8.48,8.93


**df3 => export from Sports Tracker app**

In [6]:
gpx_data_list = []

for filename in os.listdir(workouts_folder_path3):
    if filename.endswith(".gpx"):
        gpx_file_path = os.path.join(workouts_folder_path3, filename)

        with open(gpx_file_path, "r") as gpx_file:
            gpx_data = gpxpy.parse(gpx_file)

            if gpx_data.tracks and gpx_data.tracks[0].segments and gpx_data.tracks[0].segments[0].points:
                start_time = gpx_data.tracks[0].segments[0].points[0].time

                date = start_time.strftime("%Y-%m-%d")
                start_time = start_time.strftime("%H:%M:%S")
                end_time = gpx_data.tracks[0].segments[0].points[-1].time
                duration_seconds = int((end_time - gpx_data.tracks[0].segments[0].points[0].time).total_seconds())

                distance_km = round((gpx_data.length_3d() / 1000), 2)
                speed_avg_kmh = round(distance_km / (duration_seconds / 3600), 2)

                gpx_data_list.append({
                    "sport" : "Running",
                    "source" : "Sports Tracker",
                    "title": np.nan,
                    "date": date,
                    "start_time": start_time,
                    "duration_s": duration_seconds,
                    "distance_km": distance_km,
                    "speed_avg_kmh": speed_avg_kmh
                })

df3 = pd.DataFrame(gpx_data_list)
df3['date'] = pd.to_datetime(df3['date'])

display(df3)


,sport,source,title,date,start_time,duration_s,distance_km,speed_avg_kmh
0,Running,Sports Tracker,NaN,2021-01-28,17:53:47,3392,10.09,10.71
1,Running,Sports Tracker,NaN,2021-01-23,10:51:01,3421,10.15,10.68
2,Running,Sports Tracker,NaN,2021-01-15,16:43:11,848,2.31,9.81
3,Running,Sports Tracker,NaN,2021-01-10,16:34:57,3651,10.11,9.97
4,Running,Sports Tracker,NaN,2020-12-27,07:58:19,3568,10.11,10.20
5,Running,Sports Tracker,NaN,2020-12-31,09:12:35,5369,15.32,10.27
6,Running,Sports Tracker,NaN,2020-12-23,11:28:26,4501,12.19,9.75
7,Running,Sports Tracker,NaN,2020-12-21,16:34:33,3170,9.32,10.58
8,Running,Sports Tracker,NaN,2020-12-19,08:05:39,2567,6.89,9.66
9,Running,Sports Tracker,NaN,2020-12-01,19:06:07,3538,10.25,10.43


**df4 => export from Garmin Connect till the end of 2023**

In [7]:
def convert_pace_to_speed(pace):
    pace_in_seconds = int(pace.split(':')[0]) * 60 + int(pace.split(':')[1])
    speed_in_kph = 3600 / pace_in_seconds
    return speed_in_kph

df4 = pd.read_csv(workouts_folder_path4, skip_blank_lines=False)
df4['source'] = 'Garmin Connect'
df4['Typ aktywności'] = df4['Typ aktywności'].replace('Bieganie', 'Running')
df4['Czas'] = pd.to_timedelta(df4['Czas']).dt.total_seconds()
df4['Czas'] = df4['Czas'].astype('int64')

df4['Data'] = pd.to_datetime(df4['Data'])
df4['start_time'] = df4['Data'].dt.time
df4['Data'] = df4['Data'].dt.date

df4.rename(columns={'Typ aktywności': 'sport', "Data": 'date', "Czas": "duration_s", "Dystans": "distance_km", "Średnie tempo": "speed_avg_kmh", "Tytuł": "title"}, inplace=True)
df4['speed_avg_kmh'] = df4['speed_avg_kmh'].apply(convert_pace_to_speed).round(2)
df4 = df4[['sport', 'source', 'title', 'date', 'start_time', 'duration_s', 'distance_km', 'speed_avg_kmh']]
df4['date'] = pd.to_datetime(df4['date'])
df4['title'] = df4['title'].apply(lambda x: np.nan if x == 'Bieganie' else x)

display(df4)

,sport,source,title,date,start_time,duration_s,distance_km,speed_avg_kmh
0,Running,Garmin Connect,Olsztyn Bieganie,2023-11-05,09:38:17,2029,5.65,10.03
1,Running,Garmin Connect,NaN,2023-10-01,12:21:42,3002,7.37,8.85
2,Running,Garmin Connect,NaN,2023-09-05,19:44:07,3133,8.06,9.25
3,Running,Garmin Connect,NaN,2023-09-03,18:43:23,584,1.40,8.61
4,Running,Garmin Connect,Poland Business Run 2023 (wirtualnie),2023-09-03,18:16:03,1276,4.02,11.32
...,...,...,...,...,...,...,...,...
146,Running,Garmin Connect,Bieg Mikołajów 2017,2017-12-02,12:28:00,3515,12.50,12.81
147,Running,Garmin Connect,Bieg Niepodległości z PKO Bankiem Polskim,2016-11-11,15:12:00,2808,10.00,12.81
148,Running,Garmin Connect,I Spartakiada Samorządowa,2016-08-26,11:15:00,1524,5.80,13.69
149,Running,Garmin Connect,III Bieg Uniwersytecki,2016-06-04,12:00:00,2767,10.00,13.00


**df12 => merged df1 with df2**

In [8]:
merged_df = pd.merge(df2, df1, on=['date', 'distance_km'], how='left')

df12 = pd.DataFrame()

df12['sport'] = merged_df['sport_x']
df12['source'] = merged_df['source_x'] + ', ' + merged_df['source_y']
df12['date'] = merged_df['date']
df12['start_time'] = merged_df['start_time']
df12['end_time'] = merged_df['end_time']
df12['distance_km'] = merged_df['distance_km']
df12['duration_s'] = merged_df['duration_s_x'].combine_first(merged_df['duration_s_y'])
df12['speed_avg_kmh'] = merged_df['speed_avg_kmh_x'].combine_first(merged_df['speed_avg_kmh_y'])
df12['mismatch'] = ''

mask_duration_not_nan = ~merged_df['duration_s_x'].isna() & ~merged_df['duration_s_y'].isna()
mask_speed_not_nan = ~merged_df['speed_avg_kmh_x'].isna() & ~merged_df['speed_avg_kmh_y'].isna()

tolerance_speed = 0.01

duration_mismatch = mask_duration_not_nan & (merged_df['duration_s_x'] != merged_df['duration_s_y'])
speed_mismatch = mask_speed_not_nan & (abs(merged_df['speed_avg_kmh_x'] - merged_df['speed_avg_kmh_y']) > tolerance_speed)

df12.loc[duration_mismatch, 'mismatch'] += 'duration_s; '
df12.loc[speed_mismatch, 'mismatch'] += f'speed_avg_kmh, {tolerance_speed}, values ({merged_df.loc[speed_mismatch, "speed_avg_kmh_x"].astype(str)}, {merged_df.loc[speed_mismatch, "speed_avg_kmh_y"].astype(str)}); '

mismatches = df12[df12['mismatch'] != '']


if not mismatches.empty:
  display(mismatches)
else:
  df12 = df12.drop('mismatch', axis=1)
  df12['title'] = np.nan
  display(df12)


,sport,source,date,start_time,end_time,distance_km,duration_s,speed_avg_kmh,title
0,Running,NaN,2020-12-17,NaN,NaN,1.61,600,9.66,NaN
1,Running,"Map My Run app, Endomondo (Manual)",2020-11-08,17:29:00,17:29:03,9.16,3163,10.43,NaN
2,Running,"Map My Run app, Endomondo (GPS)",2020-11-04,19:53:52,19:53:55,9.37,3370,10.01,NaN
3,Running,"Map My Run app, Endomondo (GPS)",2020-11-01,16:52:07,16:52:10,9.44,3239,10.49,NaN
4,Running,"Map My Run app, Endomondo (GPS)",2020-10-29,14:59:15,16:00:49,10.03,3673,9.83,NaN
...,...,...,...,...,...,...,...,...,...
713,Running,"Map My Run app, Endomondo (Manual)",2012-08-26,18:21:00,19:13:00,7.77,3120,8.97,NaN
714,Running,"Map My Run app, Endomondo (Manual)",2012-08-22,18:30:00,19:24:00,7.77,3240,8.63,NaN
715,Running,"Map My Run app, Endomondo (Manual)",2012-08-20,18:30:00,19:19:00,6.73,2940,8.24,NaN
716,Running,"Map My Run app, Endomondo (Manual)",2012-08-18,18:07:00,19:04:00,8.48,3420,8.93,NaN


**df123 => merged df12 with df3**

In [9]:
rows_to_remove = df3[df3['date'] < '2021-12-19'].index
df3 = df3.drop(rows_to_remove)

df123 = pd.concat([df12, df3], ignore_index=True)
df123 = df123[['sport', 'source', 'title', 'date', 'start_time', 'end_time', 'distance_km', 'duration_s', 'speed_avg_kmh']]
df123 = df123.sort_values(by='date')
display(df123)

,sport,source,title,date,start_time,end_time,distance_km,duration_s,speed_avg_kmh
717,Running,"Map My Run app, Endomondo (Manual)",NaN,2012-08-15,10:07:00,10:33:00,4.21,1560,9.72
716,Running,"Map My Run app, Endomondo (Manual)",NaN,2012-08-18,18:07:00,19:04:00,8.48,3420,8.93
715,Running,"Map My Run app, Endomondo (Manual)",NaN,2012-08-20,18:30:00,19:19:00,6.73,2940,8.24
714,Running,"Map My Run app, Endomondo (Manual)",NaN,2012-08-22,18:30:00,19:24:00,7.77,3240,8.63
713,Running,"Map My Run app, Endomondo (Manual)",NaN,2012-08-26,18:21:00,19:13:00,7.77,3120,8.97
...,...,...,...,...,...,...,...,...,...
4,Running,"Map My Run app, Endomondo (GPS)",NaN,2020-10-29,14:59:15,16:00:49,10.03,3673,9.83
3,Running,"Map My Run app, Endomondo (GPS)",NaN,2020-11-01,16:52:07,16:52:10,9.44,3239,10.49
2,Running,"Map My Run app, Endomondo (GPS)",NaN,2020-11-04,19:53:52,19:53:55,9.37,3370,10.01
1,Running,"Map My Run app, Endomondo (Manual)",NaN,2020-11-08,17:29:00,17:29:03,9.16,3163,10.43


**df1234 => merged df123 with df4**

In [10]:


df1234 = pd.concat([df123, df4], ignore_index=True)
df1234 = df1234.sort_values(by='date')
display(df1234)


,sport,source,title,date,start_time,end_time,distance_km,duration_s,speed_avg_kmh
0,Running,"Map My Run app, Endomondo (Manual)",NaN,2012-08-15,10:07:00,10:33:00,4.21,1560,9.72
1,Running,"Map My Run app, Endomondo (Manual)",NaN,2012-08-18,18:07:00,19:04:00,8.48,3420,8.93
2,Running,"Map My Run app, Endomondo (Manual)",NaN,2012-08-20,18:30:00,19:19:00,6.73,2940,8.24
3,Running,"Map My Run app, Endomondo (Manual)",NaN,2012-08-22,18:30:00,19:24:00,7.77,3240,8.63
4,Running,"Map My Run app, Endomondo (Manual)",NaN,2012-08-26,18:21:00,19:13:00,7.77,3120,8.97
...,...,...,...,...,...,...,...,...,...
722,Running,Garmin Connect,Poland Business Run 2023 (wirtualnie),2023-09-03,18:16:03,NaN,4.02,1276,11.32
721,Running,Garmin Connect,NaN,2023-09-03,18:43:23,NaN,1.40,584,8.61
720,Running,Garmin Connect,NaN,2023-09-05,19:44:07,NaN,8.06,3133,9.25
719,Running,Garmin Connect,NaN,2023-10-01,12:21:42,NaN,7.37,3002,8.85


In [11]:
duplicate_rows = df1234[df1234.duplicated(subset=['date', 'duration_s', 'distance_km'], keep=False)]

if not duplicate_rows.empty:
    count = int(len(duplicate_rows) / 2)
    print(f"{count} duplicated rows to be deleted")
    y = input("Confirm deletion ")

    if y.upper() == 'Y':
        condition = pd.isna(duplicate_rows['title'])
        df1234 = df1234[~df1234.index.isin(duplicate_rows[condition].index)]
        print(f"Deleted {count} rows.")
        duplicate_rows = df1234[df1234.duplicated(subset=['date', 'duration_s', 'distance_km'], keep=False)]
        if not duplicate_rows.empty:
          count = int(len(duplicate_rows) / 2)
          print(f"fOUND {count} duplicated rows.")
        else:
          print("No duplicated rows.")
          display(df1234)
else:
    print("No duplicated rows.")
    display(df1234)


16 duplicated rows to be deleted
Confirm deletion y
Deleted 16 rows.
No duplicated rows.


,sport,source,title,date,start_time,end_time,distance_km,duration_s,speed_avg_kmh
0,Running,"Map My Run app, Endomondo (Manual)",NaN,2012-08-15,10:07:00,10:33:00,4.21,1560,9.72
1,Running,"Map My Run app, Endomondo (Manual)",NaN,2012-08-18,18:07:00,19:04:00,8.48,3420,8.93
2,Running,"Map My Run app, Endomondo (Manual)",NaN,2012-08-20,18:30:00,19:19:00,6.73,2940,8.24
3,Running,"Map My Run app, Endomondo (Manual)",NaN,2012-08-22,18:30:00,19:24:00,7.77,3240,8.63
4,Running,"Map My Run app, Endomondo (Manual)",NaN,2012-08-26,18:21:00,19:13:00,7.77,3120,8.97
...,...,...,...,...,...,...,...,...,...
722,Running,Garmin Connect,Poland Business Run 2023 (wirtualnie),2023-09-03,18:16:03,NaN,4.02,1276,11.32
721,Running,Garmin Connect,NaN,2023-09-03,18:43:23,NaN,1.40,584,8.61
720,Running,Garmin Connect,NaN,2023-09-05,19:44:07,NaN,8.06,3133,9.25
719,Running,Garmin Connect,NaN,2023-10-01,12:21:42,NaN,7.37,3002,8.85


**Checks**

In [12]:
df1234['year_month'] = df1234['date'].dt.to_period('M')

result1 = df1234.groupby('year_month')['distance_km'].sum().reset_index()
result1['year_month'] = result1['year_month'].dt.to_timestamp().dt.strftime('%Y-%m')
display(result1)

<ipython-input-12-0f6ec1ec15ab>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1234['year_month'] = df1234['date'].dt.to_period('M')


,year_month,distance_km
0,2012-08,52.35
1,2012-09,98.59
2,2012-10,91.96
3,2012-11,95.55
4,2012-12,52.71
...,...,...
123,2023-07,49.16
124,2023-08,23.22
125,2023-09,13.48
126,2023-10,7.37


In [13]:

google_sheets_link = 'https://docs.google.com/spreadsheets/d/1sKQRBWY5xvY-erhFxXo0bC0pTpZT6QUUMx1fjLngGk0/edit?usp=drive_link'

document_key = google_sheets_link.split('/')[-2]

worksheet_link = f'https://docs.google.com/spreadsheets/d/{document_key}/export?format=csv'

dfgoogle = pd.read_csv(worksheet_link, header=1)
dfgoogle = dfgoogle.drop(columns=['Unnamed: 0'])
dfgoogle = dfgoogle.iloc[:, :-1]
dfgoogle = dfgoogle.iloc[:-1,:]
dfgoogle.replace('-', np.nan, inplace=True)
display(dfgoogle)


,miesiąc / rok,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,styczeń,NaN,"76,59","43,08","48,17","65,66","51,16","106,34","0,00","40,75","32,10","47,98","0,00"
1,luty,NaN,"57,04","65,36","83,30","44,66","46,28","17,88","7,79","67,29","15,09","47,70","8,14"
2,marzec,NaN,"41,12","44,55","90,17","138,11","38,07","43,73","48,15","94,18","0,00","18,01","12,17"
3,kwiecień,NaN,"58,38","52,41","64,98","109,17","68,11","73,62","45,31","111,77","72,62","29,20","0,00"
4,maj,NaN,"83,13","89,69","156,73","149,46","39,31","45,77","71,53","108,07","104,15","29,55","26,97"
5,czerwiec,NaN,"77,39","116,45","83,30","94,79","36,29","32,15","36,45","74,35","79,11","10,06","33,88"
6,lipiec,NaN,"73,47","104,91","70,30","93,11","51,82","66,03","77,27","124,15","111,90","0,00","49,16"
7,sierpień,"52,35","51,63","172,00","76,41","66,36","89,59","52,94","47,96","131,90","126,96","5,02","23,22"
8,wrzesień,"98,59","87,30","106,57","104,72","39,38","30,54","60,32","58,86","138,45","120,55","5,06","13,48"
9,październik,"91,96","62,25","73,01","113,72","28,00","69,00","37,71","64,69","50,10","83,80","0,00",NaN


In [14]:
result2 = pd.DataFrame(columns=['year_month', 'distance_km'])

for index, row in dfgoogle.iterrows():
    for col in dfgoogle.columns[1:]:
        year_month = f'{col}-{index + 1:02d}'  # Using col as a string
        distance_km = row[col]
        result2 = pd.concat([result2, pd.DataFrame({'year_month': [year_month], 'distance_km': [distance_km]})], ignore_index=True)

result2 = result2.sort_values(by='year_month')
result2 = result2[result2['year_month'].apply(lambda x: int(x.split('-')[1])) != 13]

result2['distance_km'] = result2['distance_km'].str.replace(',', '.').astype(float)
result2['distance_km'] = pd.to_numeric(result2['distance_km'])

display(result2)


,year_month,distance_km
0,2012-01,NaN
12,2012-02,NaN
24,2012-03,NaN
36,2012-04,NaN
48,2012-05,NaN
...,...,...
95,2023-08,23.22
107,2023-09,13.48
119,2023-10,NaN
131,2023-11,NaN


In [15]:
result1 = result1.rename(columns={'distance_km': 'result1_distance_km'})
result2 = result2.rename(columns={'distance_km': 'result2_distance_km'})

result = pd.concat([result1.set_index('year_month'), result2.set_index('year_month')], axis=1, join='outer')

result['absolute_difference'] = abs(result['result2_distance_km'] - result['result1_distance_km'])

print(result)

            result1_distance_km  result2_distance_km  absolute_difference
year_month                                                               
2012-08                   52.35                52.35                  0.0
2012-09                   98.59                98.59                  0.0
2012-10                   91.96                91.96                  0.0
2012-11                   95.55                95.55                  0.0
2012-12                   52.71                52.71                  0.0
...                         ...                  ...                  ...
2022-10                     NaN                 0.00                  NaN
2022-11                     NaN                 0.00                  NaN
2023-01                     NaN                 0.00                  NaN
2023-04                     NaN                 0.00                  NaN
2023-12                     NaN                  NaN                  NaN

[144 rows x 3 columns]


In [16]:
filtered_df = result[result['absolute_difference'] > 0.02].copy()
print(filtered_df)

            result1_distance_km  result2_distance_km  absolute_difference
year_month                                                               
2017-06                   36.27                36.29                 0.02
2018-05                   45.75                45.77                 0.02
2018-06                   42.95                32.15                10.80
2020-01                   11.21                40.75                29.54
2020-11                   27.97                74.58                46.61
2020-12                   54.91                63.38                 8.47
2021-08                  127.11               126.96                 0.15
2021-09                  120.99               120.55                 0.44


In [17]:
selected_rows = df1234[(df1234['date'] >= '2020-12-19') & (df1234['date'] <= '2021-01-31')]

display(selected_rows)

,sport,source,title,date,start_time,end_time,distance_km,duration_s,speed_avg_kmh,year_month
850,Running,Garmin Connect,NaN,2020-12-19,09:05:00,NaN,6.81,2500,9.81,2020-12
849,Running,Garmin Connect,NaN,2020-12-21,17:34:00,NaN,9.25,3117,10.68,2020-12
848,Running,Garmin Connect,NaN,2020-12-23,12:28:00,NaN,12.06,4402,9.86,2020-12
847,Running,Garmin Connect,NaN,2020-12-27,08:58:19,NaN,10.01,3531,10.20,2020-12
846,Running,Garmin Connect,NaN,2020-12-31,10:12:35,NaN,15.17,5321,10.26,2020-12
845,Running,Garmin Connect,NaN,2021-01-10,17:34:57,NaN,10.01,3573,10.08,2021-01
844,Running,Garmin Connect,NaN,2021-01-15,17:43:11,NaN,2.32,849,9.84,2021-01
843,Running,Garmin Connect,NaN,2021-01-23,11:51:01,NaN,9.75,3339,10.53,2021-01
842,Running,Garmin Connect,NaN,2021-01-28,18:53:47,NaN,10.03,3346,10.78,2021-01
